In [1]:
import pandas as pd

import numpy as np
import matplotlib.pyplot as plt

In [2]:
dataset = pd.read_csv('Social_Network_Ads.csv')

In [3]:
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [4]:
dataset = pd.get_dummies(dataset,drop_first = True)

In [5]:
dataset

,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,1
1,15810944,35,20000,0,1
2,15668575,26,43000,0,0
3,15603246,27,57000,0,0
4,15804002,19,76000,0,1
...,...,...,...,...,...
395,15691863,46,41000,1,0
396,15706071,51,23000,1,1
397,15654296,50,20000,1,0
398,15755018,36,33000,0,1


In [6]:
dataset.columns

Index(['User ID', 'Age', 'EstimatedSalary', 'Purchased', 'Gender_Male'], dtype='object')

In [7]:
dataset = dataset.drop('User ID' , axis = 1)

In [8]:
dataset

,Age,EstimatedSalary,Purchased,Gender_Male
0,19,19000,0,1
1,35,20000,0,1
2,26,43000,0,0
3,27,57000,0,0
4,19,76000,0,1
...,...,...,...,...
395,46,41000,1,0
396,51,23000,1,1
397,50,20000,1,0
398,36,33000,0,1


In [9]:
dataset['Purchased'].value_counts()

0    257
1    143
Name: Purchased, dtype: int64

In [10]:
indep = dataset[['Age', 'EstimatedSalary', 'Gender_Male']]

dep = dataset['Purchased']

In [11]:
indep.shape

(400, 3)

In [12]:
indep

,Age,EstimatedSalary,Gender_Male
0,19,19000,1
1,35,20000,1
2,26,43000,0
3,27,57000,0
4,19,76000,1
...,...,...,...
395,46,41000,0
396,51,23000,1
397,50,20000,0
398,36,33000,1


In [13]:
dep

0      0
1      0
2      0
3      0
4      0
      ..
395    1
396    1
397    1
398    0
399    1
Name: Purchased, Length: 400, dtype: int64

In [14]:
from sklearn.model_selection import train_test_split

X_train,X_test,Y_train,Y_test = train_test_split(indep,dep, test_size = 0.30, random_state = 0)

In [15]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [16]:
from sklearn.tree import DecisionTreeClassifier

from sklearn.model_selection import GridSearchCV

param_grid = {'criterion' : ['gini','entropy'],
              'splitter': ['best', 'random'],
              'max_features': ['auto', 'sqrt','log2']
              }
grid = GridSearchCV(DecisionTreeClassifier(),param_grid,refit = 'True', verbose = 3,cv = 5,n_jobs = -1, scoring = 'f1_weighted')
#fiiting the model for grid
grid.fit(X_train,Y_train)



Fitting 5 folds for each of 12 candidates, totalling 60 fits


GridSearchCV(cv=5, estimator=DecisionTreeClassifier(), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_features': ['auto', 'sqrt', 'log2'],
                         'splitter': ['best', 'random']},
             refit='True', scoring='f1_weighted', verbose=3)

In [17]:
#print best parameter after tuning 
#print(grid.best_params_)

re = grid.cv_results_

grid_predictions = grid.predict(X_test)



from sklearn.metrics import confusion_matrix
cm = confusion_matrix (Y_test,grid_predictions)

#print classification report

from sklearn.metrics import classification_report

clf_report  = classification_report(Y_test,grid_predictions)

In [18]:
from sklearn.metrics import f1_score

f1_macro = f1_score(Y_test,grid_predictions,average = 'weighted')

print('The R_score vlaue for best parameter {}:'.format(grid.best_params_),f1_score)

The R_score vlaue for best parameter {'criterion': 'entropy', 'max_features': 'sqrt', 'splitter': 'random'}: <function f1_score at 0x00000249700AB288>


In [19]:
print (cm)

[[72  7]
 [ 7 34]]


In [20]:
print (clf_report)

              precision    recall  f1-score   support

           0       0.91      0.91      0.91        79
           1       0.83      0.83      0.83        41

    accuracy                           0.88       120
   macro avg       0.87      0.87      0.87       120
weighted avg       0.88      0.88      0.88       120



In [21]:
from sklearn.metrics import roc_auc_score

roc_auc_score(Y_test,grid.predict_proba(X_test)[:,-1])

0.8703303488731089

In [22]:
table = pd.DataFrame.from_dict(re)

In [23]:
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_features,param_splitter,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.006596,0.001356,0.006806,0.001707,gini,auto,best,"{'criterion': 'gini', 'max_features': 'auto', ...",0.835985,0.823129,0.789152,0.874356,0.891667,0.842858,0.036593,3
1,0.029935,0.030873,0.008994,0.001548,gini,auto,random,"{'criterion': 'gini', 'max_features': 'auto', ...",0.802399,0.804584,0.858503,0.735075,0.891667,0.818445,0.053603,11
2,0.005599,0.001022,0.008794,0.001939,gini,sqrt,best,"{'criterion': 'gini', 'max_features': 'sqrt', ...",0.835985,0.769053,0.804584,0.822861,0.909115,0.828319,0.046244,6
3,0.005805,0.000753,0.007789,0.001712,gini,sqrt,random,"{'criterion': 'gini', 'max_features': 'sqrt', ...",0.766471,0.821429,0.787755,0.819444,0.891667,0.817353,0.042446,12
4,0.116641,0.054507,0.009794,0.001164,gini,log2,best,"{'criterion': 'gini', 'max_features': 'log2', ...",0.835985,0.746799,0.824293,0.892857,0.836407,0.827268,0.046788,8
5,0.004601,0.000800,0.006794,0.001601,gini,log2,random,"{'criterion': 'gini', 'max_features': 'log2', ...",0.852966,0.874254,0.746799,0.875548,0.872761,0.844465,0.049528,2
6,0.006193,0.000401,0.008798,0.002312,entropy,auto,best,"{'criterion': 'entropy', 'max_features': 'auto...",0.891485,0.857143,0.806153,0.804432,0.833784,0.838600,0.032815,4
7,0.004803,0.000404,0.008193,0.000989,entropy,auto,random,"{'criterion': 'entropy', 'max_features': 'auto...",0.855314,0.819142,0.769053,0.839990,0.853485,0.827397,0.031908,7
8,0.004997,0.000633,0.007194,0.001715,entropy,sqrt,best,"{'criterion': 'entropy', 'max_features': 'sqrt...",0.821429,0.819142,0.772092,0.857143,0.826525,0.819266,0.027267,10
9,0.005997,0.000893,0.008594,0.001624,entropy,sqrt,random,"{'criterion': 'entropy', 'max_features': 'sqrt...",0.795918,0.855314,0.841398,0.876841,0.891667,0.852228,0.033042,1
